# 📅 PyKwant Tutorial: Date Handling and Calendars
This notebook explores the `pykwant.dates` module, designed to handle the complexities of financial calendars, business day conventions, and day count calculations using a functional paradigm.

## 1. Setup and Imports
We import the necessary dependencies and the dates module from our library.

In [5]:
from datetime import date
from pykwant import dates

# Helper for clean output display
def print_date_info(label: str, d: date):
    print(f"{label}: {d} ({d.strftime('%A')})")

## 2. Defining a Calendar
In `pykwant`, a calendar is an immutable data structure (`frozen dataclass`). Let's define a custom calendar (e.g., "Milan") with specific holidays.

> **Technical Note**: The `holidays` field is a `frozenset` to ensure O(1) lookup performance and immutability.

In [6]:
# Define some holidays (New Year's, Epiphany, Christmas)
holidays = frozenset([
    date(2025, 1, 1),   # New Year's Day
    date(2025, 1, 6),   # Epiphany (Monday)
    date(2025, 12, 25), # Christmas
    date(2025, 12, 26)  # St. Stephen's Day
])

# Create the Calendar object. Default weekends=(SATURDAY, SUNDAY)
cal_milano = dates.Calendar(holidays=holidays)

print(f"Calendar created with {len(cal_milano.holidays)} holidays.")

Calendar created with 4 holidays.


### Checking Business Days
We use the pure function `is_business_day` to verify if a date is a working day.

In [7]:
test_dates = [
    date(2025, 1, 1),  # Holiday
    date(2025, 1, 4),  # Saturday
    date(2025, 1, 6),  # Monday (but Holiday - Epiphany)
    date(2025, 1, 7),  # Tuesday (Business Day)
]

print("--- Business Day Check ---")
for d in test_dates:
    is_biz = dates.is_business_day(d, cal_milano)
    status = "Business Day" if is_biz else "Non-Business Day"
    print_date_info(f"[{status}]", d)

--- Business Day Check ---
[Non-Business Day]: 2025-01-01 (Wednesday)
[Non-Business Day]: 2025-01-04 (Saturday)
[Non-Business Day]: 2025-01-06 (Monday)
[Business Day]: 2025-01-07 (Tuesday)


## 3. Rolling Conventions (Date Adjustment)
When a payment date falls on a non-business day, it must be "rolled" (adjusted). `pykwant` implements various conventions as functions: `RollingConvention = Callable[[date, Calendar], date]`.

Let's examine the case of **January 4, 2025** (Saturday) and **January 6, 2025** (Monday, Holiday).

In [8]:
target_date = date(2025, 1, 4) # Saturday
print_date_info("Original Date", target_date)

# 1. Following: First business day after
d_foll = dates.following(target_date, cal_milano)
print_date_info("Following", d_foll) 
# Should skip Sat(4), Sun(5), Mon(6-Epiphany) -> Tue(7)

# 2. Preceding: First business day before
d_prec = dates.preceding(target_date, cal_milano)
print_date_info("Preceding", d_prec)
# Should return to Fri(3)

Original Date: 2025-01-04 (Saturday)
Following: 2025-01-07 (Tuesday)
Preceding: 2025-01-03 (Friday)


### Modified Following
This is the most common convention for derivatives. It acts like `Following`, but if the roll pushes the date into the next month, it uses `Preceding` to stay in the current month.

In [9]:
# Special case: Month-end falling on a holiday/weekend.
# Example: May 31, 2025 is a Saturday.
d_end_month = date(2025, 5, 31) 
print_date_info("\nMonth End (Saturday)", d_end_month)

d_mf = dates.modified_following(d_end_month, cal_milano)
print_date_info("Modified Following", d_mf)
# 'Following' would go to June (1st or 2nd). 
# 'Modified Following' must return to Friday, May 30th.


Month End (Saturday): 2025-05-31 (Saturday)
Modified Following: 2025-05-30 (Friday)


## 4. Day Count Conventions (Year Fractions)
The `DayCountConvention` functions calculate the year fraction between two dates, which is essential for accruing interest and calculating discount factors.

In [10]:
start = date(2025, 1, 1)
end = date(2025, 4, 1) # Exactly 90 calendar days (31+28+31)

print(f"Period: {start} -> {end}")

# ACT/365
tau_act365 = dates.act_365(start, end)
print(f"ACT/365:    {tau_act365:.6f} (={end-start} days / 365)")

# ACT/360
tau_act360 = dates.act_360(start, end)
print(f"ACT/360:    {tau_act360:.6f} (={end-start} days / 360)")

# 30/360 (Bond Basis)
# Assumes 30-day months
tau_30360 = dates.thirty_360(start, end)
print(f"30/360:     {tau_30360:.6f}  (=90 days / 360)")

Period: 2025-01-01 -> 2025-04-01
ACT/365:    0.246575 (=90 days, 0:00:00 days / 365)
ACT/360:    0.250000 (=90 days, 0:00:00 days / 360)
30/360:     0.250000  (=90 days / 360)


## 5. Schedule Generation
The `generate_schedule` function combines everything we've seen (calendars and conventions) to generate payment dates for a financial instrument (e.g., Swap or Bond).

**Scenario**: An annual Bond paying coupons every 3 months, starting January 1, 2025.

In [11]:
schedule = dates.generate_schedule(
    start=date(2025, 1, 1),
    end=date(2026, 1, 1),
    freq_month=3,               # Quarterly
    cal=cal_milano,             # Our calendar
    convention=dates.modified_following # Rolling convention
)

print("\n--- Generated Schedule (Quarterly, Mod. Following) ---")
for i, d in enumerate(schedule):
    print(f"Date {i+1}: {d} ({d.strftime('%A')})")

# Note: 
# Theoretical Date 1: 2025-04-01 (Tuesday) -> OK
# Theoretical Date 2: 2025-07-01 (Tuesday) -> OK
# Theoretical Date 3: 2025-10-01 (Wednesday) -> OK
# Theoretical Date 4: 2026-01-01 (Thursday, HOLIDAY) -> Shift to Friday 2026-01-02


--- Generated Schedule (Quarterly, Mod. Following) ---
Date 1: 2025-01-02 (Thursday)
Date 2: 2025-04-01 (Tuesday)
Date 3: 2025-07-01 (Tuesday)
Date 4: 2025-10-01 (Wednesday)
Date 5: 2026-01-01 (Thursday)
